In [2]:
! git clone https://github.com/biplav-s/image-mask-estimate

Cloning into 'image-mask-estimate'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 37 (delta 2), reused 31 (delta 1), pack-reused 0
Unpacking objects: 100% (37/37), done.


In [3]:
!pip install lime

     |████████████████████████████████| 276kB 10.3MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp36-none-any.whl size=283846 sha256=aa392d0a982f194967e69811f33e7c5a31c5833f8313b8ac0351bcf54703a8f7
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime


In [ ]:
!pip install fdet

In [5]:
import torch
from lime import lime_image
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from glob import glob
from skimage.segmentation import mark_boundaries
from fdet import io, MTCNN

In [37]:
#for controlling max-image opening through matplotlib in colab
import matplotlib as mpl
mpl.rc('figure', max_open_warning = 0)

In [6]:
# downloading yolo weight of our trained model
from google.colab import files
files.upload()

Saving best.pt to best.pt


In [32]:
model = torch.hub.load('ultralytics/yolov5','custom', path_or_model='/content/best.pt')
detector1 = model.autoshape() 

detector2 = MTCNN()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                   

Adding autoShape... 
autoShape already enabled, skipping... 


In [33]:
#lime explainer
explainer = lime_image.LimeImageExplainer()
def lime_result(model,batch_predict,paths,num_samples=200,num_features=10): 
  
  for i,path in enumerate(paths):
    img = Image.open(path)
    img = img.resize((224,224))
    
    explanation = explainer.explain_instance(np.array(img),batch_predict , top_labels=5, hide_color=0, num_samples=num_samples)
    
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=num_features, hide_rest=False)
    img_boundry1 = mark_boundaries(temp/255.0, mask)
    
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=num_features, hide_rest=False)
    img_boundry2 = mark_boundaries(temp/255.0, mask)
    
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(img_boundry1)
    axarr[1].imshow(img_boundry2)
    plt.savefig(f'{model}_{i}.jpg')

In [34]:
paths = glob('/content/image-mask-estimate/data/sample-images/*')

In [35]:
def Yolo_sum(images):
    pred = []
    for image in images:
      sum=0
      logits = detector1(image)
      result = np.array(logits.xyxy[0][:,4])
      for i in result:
        sum+=i
      sum = sum/len(result) if len(result) else 0 
      pred.append([sum,1-sum])
    return np.array(pred)

In [38]:
lime_result('Yolo_sum',Yolo_sum,paths,num_samples=10)

In [39]:
def Yolo_multi(images):
    pred = []
    for num,image in enumerate(images):
      results = []
      logits = detector1(image)
      result = np.array(logits.xyxy[0][:,4])
      for i in result:
        results.append(i)
        results.append(1-i)  
      pred.append(results)
    np_result = np.zeros((images.shape[0],100))
    for i in range(images.shape[0]):
      size = len(pred[i]) if len(pred[i])<=100 else 100 
      np_result[i,:size] = pred[i][:size] 
    return np_result

In [40]:
lime_result('Yolo_multi',Yolo_multi,paths,num_samples=10)

In [41]:
def MTCNN_sum(images):
    pred = []
    for image in images:
      sum=0
      logits = detector2.detect(image)
      for i in logits:
        sum+=i['confidence']
      sum = sum/len(logits) if len(logits) else 0 
      pred.append([sum,1-sum])
    return np.array(pred)

In [42]:
lime_result('MTCNN_sum',MTCNN_sum,paths,num_samples=10)

In [43]:
def MTCNN_multi(images):
    pred = []
    for num,image in enumerate(images):
      results = []
      logits = detector2.detect(image)
      for i in logits:
        results.append(i['confidence'])
        results.append(1-i['confidence'])  
      pred.append(results)
    np_result = np.zeros((images.shape[0],100))
    for i in range(images.shape[0]):
      size = len(pred[i]) if len(pred[i])<=100 else 100 
      np_result[i,:size] = pred[i][:size] 
    return np_result

In [44]:
lime_result('MTCNN_multi',MTCNN_sum,paths,num_samples=10)